# Библиотеки

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import time


from deap import base, algorithms
from deap import creator
from deap import tools

# Принцип работы

Создание начальной популяции -> цикл{вычисление приспособленности->отбор->скрещивание->мутация->провеска условий останова}->выбор лучшего


В отборе оставляют не только самых лучших.

    Турнирный отбор
    
    Из популяции отбираютсяслучайные, а из них выбирают лучшего. Лучший будет партнером для всех

Кроссингровер - скрщивание

# Задача OneMax

Популяция - списки с 0 и 1

In [18]:
%matplotlib inline

In [19]:
def clone(value):
    ind = Individual(value[:])
    ind.fitness.values[0] = value.fitness.values[0]
    return ind


def sel_tour(population, p_len):
    offspring = []
    for n in range(p_len):
        i1 = i2 = i3 = 0
        while i1 == i2 or i1 == i3 or i2 == i3:
            i1, i2, i3 = [random.randint(0, p_len - 1) for _ in range(3)]

        offspring.append(max([population[i1], population[i2], population[i3]] , key=lambda ind: ind.fitness.values[0]))
    return offspring


def cx_one_point(child1, child2):
    s = random.randint(2, len(child1) - 3)
    child1[s:], child2[s:] = child2[s:], child1[s:]


def mutation(mutant, indpb=0.01):
    for indx in range(len(mutant)):
        if random.random() < indpb:
            mutant[indx] = 0 if mutant[indx] == 1 else 1

In [20]:
def oneMaxFitness(individual): # аля функция оценки как у меня в курсаче
    return sum(individual), 


def individualCreator():
    return Individual([random.randint(0, 1) for i in range(one_max_len)])#list(np.random.randint(0, 2, (one_max_len))))


def populationCreator(n = 0):
    return list([individualCreator() for _ in range(n)])


def cycle(one_max_len, population_size, p_crossover, p_mutation, max_generation, ):
    population = populationCreator(n=population_size)
    
    generation_counter = 0
    
    fitness_values = list(map(oneMaxFitness, population))
    
    for individual, fitness_val in zip(population, fitness_values):
        individual.fitness.values = fitness_val
    
    max_val = []
    mean_val = []
    
    fitness_values = [individual.fitness.values[0] for individual in population]
    
    while max(fitness_values) < one_max_len and generation_counter < max_generation:
        generation_counter += 1
        offspring = sel_tour(population, len(population))
        offspring = list(map(clone, offspring))
    
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < p_crossover:
                cx_one_point(child1, child2)
    
        for mutant in offspring:
            if random.random() < p_mutation:
                mutation(mutant, indpb=1.0 / one_max_len)
    
        fresh_fit_val = list(map(oneMaxFitness, offspring))
        for individual, fit_val in zip(offspring, fresh_fit_val):
            individual.fitness.values = fit_val
    
        population[:] = offspring
    
        fit_val = [ind.fitness.values[0] for ind in population]
        
        max_fit = max(fit_val)
        mean_fit = sum(fit_val) / len(fit_val)
        max_val.append(max_fit)
        mean_val.append(mean_fit)
        # print(f'Поколение {generation_counter}, МП {max_fit}, СП {mean_fit}')
    
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(np.arange(0, max_generation), max_val)#, '-.g')
    ax.plot(np.arange(0, max_generation), mean_val)#, '--.r')


In [21]:
class FitnessMax():
    def __init__(self):
        self.values = [0]


class Individual(list): # наследуемся от списка
    def __init__(self, *args):
        super().__init__(*args)
        self.fitness= FitnessMax()

In [22]:
%matplotlib tk

In [24]:
%%time

# константы
one_max_len = 100 # длина подлежай оптимизации строки

# константы генетического алгоритма
population_size = 200 # количество индивидуумов в популяции
p_crossover = 0.9 # вероятность скрещивания
p_mutation = 0.5 # вероятность мутации индивидуума
max_generation = 50 # максимальное число поколений

random_seed = 42
random.seed(random_seed)

# list(np.random.randint(0, 2, (one_max_len)))

cycle(one_max_len, population_size, p_crossover, p_mutation, max_generation)

CPU times: total: 641 ms
Wall time: 695 ms


individualCreator()

CPU times: total: 4.97 s
Wall time: 5.14 s


# ax.plot(np.arange(0, max_generation), me


CPan_val)#, '--.r')
CPU times: total: 7.81 s
Wall time: 7.98 s

In [8]:
np.random.randint(0, 2, 15)

array([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0])

In [25]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.arange(0, 50), max_val)#, '-.g')
ax.plot(np.arange(0, 50), mean_val)#, '--.r')

NameError: name 'max_val' is not defined

# Библиотека DEAP

Состоит из трех модулей - base, creator, tools

    create - позвояет создавать объекты класса

    create(Name_class, basic_class, [attribution_new_class])

    create('Point', object, max_coord=10, min_coord=10, coords=list) - после этого будет оздан класс Point

class Point(object): - неизменяемые - аттрибуты класса. Изменяемые - свойства класса
    max_coord = 10
    min_coord = 0

    def __init__(self):
        self.coords = list()

Бахнем то же самое

In [27]:
def one_max_fit(individual):
    return sum(individual), # рассчет приспособенности идивидуума

In [28]:
# %matplotlib inline
%matplotlib tk

In [30]:
one_max_len = 100 # длина подлежай оптимизации строки

# константы генетического алгоритма
population_size = 200 # количество индивидуумов в популяции
p_crossover = 0.9 # вероятность скрещивания
p_mutation = 0.1 # вероятность мутации индивидуума
max_generation = 50 # максимальное число поколений


creator.create('FitnessMax', base.Fitness, weights=(-1.0,))
# base.Fitness - не понял
# weights=(1.0) - оптимизируем одно число, причем его нужно максимизировать
creator.create('Individual', list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register('zero_or_one', random.randint, 0, 1)
# zero_or_one - псевдоним random.randint и возвращает 0 или 1
toolbox.register('individual_creator', tools.initRepeat, creator.Individual, toolbox.zero_or_one, one_max_len)
# tools.initRepeat(Контейнер для хранения генов, функция генерации значения гена, число генов в хромооме)
toolbox.register("population_creator", tools.initRepeat, list, toolbox.individual_creator)

population = toolbox.population_creator(n=population_size)

toolbox.register('evaluate', one_max_fit) # возвращает кортеж приспособенности индивидуумов
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', tools.cxOnePoint) #одноточечное скрещивание
toolbox.register('mutate', tools.mutFlipBit, indpb=1.0/one_max_len)
# эти 4 аллиаса обязательны, они предопределены

stats = tools.Statistics(lambda ind: ind.fitness.values) # какие данные индивида брать для сбора статистики
stats.register('max', np.max)
stats.register('mean', np.mean)


population, logbook = algorithms.eaSimple(population, toolbox,
                                         cxpb=p_crossover,
                                         mutpb=p_mutation,
                                         ngen=max_generation,
                                         stats=stats,
                                         verbose=True) # отражает служебную информацию

max_vals, mean_vals = logbook.select('max', 'mean')
print()
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(np.arange(max_generation + 1), max_vals, label='max')
ax.plot(np.arange(max_generation + 1), mean_vals, label='mean')
ax.legend()
plt.xlabel('Поколене')
plt.ylabel('Значение')
plt.title('Ням')

gen	nevals	max	mean  
0  	200   	60 	49.495
1  	172   	58 	44.865
2  	179   	54 	40.92 
3  	193   	46 	37.61 
4  	184   	44 	34.855
5  	186   	42 	31.73 
6  	180   	40 	29.13 
7  	193   	34 	26.21 
8  	181   	32 	23.47 
9  	185   	29 	20.905
10 	175   	25 	18.06 
11 	191   	21 	15.655
12 	184   	18 	14.12 
13 	185   	16 	12.87 
14 	191   	15 	12.105
15 	185   	15 	11.48 
16 	182   	15 	10.745
17 	178   	12 	10.005
18 	184   	13 	9.375 
19 	177   	11 	8.87  
20 	179   	12 	8.375 
21 	182   	10 	7.565 
22 	173   	12 	7.065 
23 	187   	10 	6.74  
24 	185   	8  	6.4   
25 	189   	8  	6.105 
26 	186   	9  	5.81  
27 	187   	7  	5.31  
28 	174   	8  	5.115 
29 	180   	7  	5.08  
30 	176   	9  	5.015 
31 	187   	8  	4.99  
32 	178   	9  	4.875 
33 	186   	8  	4.51  
34 	188   	9  	4.205 
35 	184   	7  	3.875 
36 	186   	7  	3.5   
37 	184   	6  	3.075 
38 	187   	6  	2.75  
39 	184   	7  	2.275 
40 	171   	4  	1.81  
41 	174   	4  	1.34  
42 	176   	4  	0.79  
43 	191   	4  	0.41  
44 	182   

Text(0.5, 1.0, 'Ням')

# Как ГА аходит решения

In [32]:
one_max_len = 100 # длина подлежай оптимизации строки

# константы генетического алгоритма
population_size = 200 # количество индивидуумов в популяции
p_crossover = 0.9 # вероятность скрещивания
p_mutation = 0.5 # вероятность мутации индивидуума
max_generation = 50 # максимальное число поколений


creator.create('FitnessMax', base.Fitness, weights=(1.0,))
# base.Fitness - не понял
# weights=(1.0) - оптимизируем одно число, причем его нужно максимизировать
creator.create('Individual', list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register('zero_or_one', random.randint, 0, 1)
# zero_or_one - псевдоним random.randint и возвращает 0 или 1
toolbox.register('individual_creator', tools.initRepeat, creator.Individual, toolbox.zero_or_one, one_max_len)
# tools.initRepeat(Контейнер для хранения генов, функция генерации значения гена, число генов в хромооме)
toolbox.register("population_creator", tools.initRepeat, list, toolbox.individual_creator)

population = toolbox.population_creator(n=population_size)

toolbox.register('evaluate', one_max_fit) # возвращает кортеж приспособенности индивидуумов
toolbox.register('select', tools.selTournament, tournsize=10)
toolbox.register('mate', tools.cxOnePoint) #одноточечное скрещивание
toolbox.register('mutate', tools.mutFlipBit, indpb=1.0/one_max_len)

# эти 4 аллиаса обязательны, они предопределены

stats = tools.Statistics(lambda ind: ind.fitness.values) # какие данные индивида брать для сбора статистики
stats.register('max', np.max)
stats.register('mean', np.mean)
stats.register('values', np.array)

population, logbook = algorithms.eaSimple(population, toolbox,
                                         cxpb=p_crossover,
                                         mutpb=p_mutation,
                                         ngen=max_generation,
                                         stats=stats,
                                         verbose=True) # отражает служебную информацию

max_vals, mean_vals, vals = logbook.select('max', 'mean', 'values')
print()
# fig, ax = plt.subplots(1, 1, figsize=(6, 4))
# ax.plot(np.arange(max_generation + 1), max_vals, label='max')
# ax.plot(np.arange(max_generation + 1), mean_vals, label='mean')
# ax.legend()
# plt.xlabel('Поколене')
# plt.ylabel('Значение')
# plt.title('Ням')


plt.ion()
fig, ax = plt.subplots()

line, = ax.plot(vals[0], ' o', markersize=1)
ax.set_ylim(40, 110)

for v in vals:
    line.set_ydata(v)

    plt.draw()
    plt.gcf().canvas.flush_events()

    time.sleep(0.5)

plt.ioff()
plt.show()

gen	nevals	max	mean 	values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

KeyboardInterrupt: 

# Методы отбора, скрещивания и мутации

## Отбор

### Правило рулетки
    Особь одбиратся с вероятностью пропорциональной приспособленности

### Стохастическая универсальная выборка 
    То же что и прошлое, но мы выбираем как бы несколько указателей выбора (представь рулетку) равномерно размещенных по окружности

### Ранжированный отбор
    Ранжируем индивидуумов по приспособленности. После что и первое, только уже выбираемномер ранга

### Масштабирование приспособленности
    Линейными преоразованиями преобразуем приспособленность к желаемому диапазону - диаграмма становится более равномерной

### Турнирный отбор
    Из популяции выбирается n индивидов. Луший из группы переходит в следующее поколение и может стать родителем


## Скрещивание

Выбирется из популяции пара родителей и с высокой вероятностью скрещиваем. Потомком переносим в следующую популяцию. Если скрещивание не прокнула, то переносим родителей в след. Популяцию

### Одноточечное скрещивание
    Выбираем место разреза генов и меняем участки хромосом родителей для потомков

### Двухточечное скрещивание
    Аналогично, но уже выбираем две точки разреза

### Равномерное скрещивание
    Выбираем рандомную хромосомину и меняем их местами. Можно менять местами не p1[1] меняем с p2[1] а другим геном т.е. p1[1] меняется на p2[4]

### Упорядоченное скрещивание
    Неля что бы числа в хромосоме дублировались. Сначала скрещиваем как хочется, оставшиеся гены родителей просматриваем. Если у p2 ген на который смотрит указатель у p1 есть у p2 вне точки разреза, то добавляем этот ген к потомку 1


### Скрещивание смешение (для чисел с плавающей точкой)
    Последовательно переберам гены родителей. На основе значений пары генов создаем диспазон в котором генерируем рандомное начение для гена потомка
$$[p1_i - \alpha(p2_i-p1_i);p2_i + \alpha(p2_i-p1_i)]$$
$$p1 <= p2$$
        Чем больше альфа тем шире новый диапазон. Обычно 0.5

### Имитация двоичного скрещивание (для чисел с плавающей точкой)
    Средняя сумма хромосом родителей и потомков равны
$$o1 or o2 = \frac{1}{2}[(1+-\beta )p1+(1-+\beta)p2$$
При $$\beta$$ == 1 хромосомы индентичны
При $$\beta$$ < 1 средние значения хромосом потомков сходятся
При $$\beta$$ > 1 средние значения хромосом потомков расходятся

Выбор $$\beta$$
$$
\begin{equation*}
\beta = 
\begin{cases}
(2u)^\frac{1}{\eta+1},u\leq0.5
\\
(\frac{1}{2}(1-u))^\frac{1}{\eta+1},u\geq0.5
\end{cases}
\end{equation*}
$$
Эта обычно либо 10 либо 20

## Мутация

### Инвертирование битов

### Мутация обменом
    Обмениваем случайно выбранными генами при этом гены не должны повторяться

### Мутация обращением / перетасовкой
    Выбираем интервал генома и случайно их перемешиваем

### Мутация для вещественных чисел
    Можно взять гауссовское распределеие со среднем == величине хромосомы на которую смотрим
    x = random.triangular(borde_left, borde_right, random.gauss(mu, sigma))



## Элитизм
    В следующее поколе запихиваем элитных особей